In [98]:
import os
#import cudf.pandas
import pandas as pd
import numpy as np
from ast import literal_eval
from collections import OrderedDict, defaultdict
import yaml
import requests
import time
#cudf.pandas.install()

os.environ['NUMEXPR_MAX_THREADS'] = '48'
os.environ['NUMEXPR_NUM_THREADS'] = '36'

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
%%time
df_pr = pd.concat([pd.read_csv('data/merged_data/filtered_github_data_large/merged_commit_pr.csv', index_col = 0),
                   pd.read_csv('data/merged_data/github_data_pre_18/merged_commit_pr.csv', index_col = 0)])

pr_cols = df_pr.columns[0:24].tolist() + df_pr.columns[47:63].tolist() + ['commit_actor_id_list']
pr_data = df_pr[pr_cols].drop_duplicates()

commit_cols = [df_pr.columns[2]] + df_pr.columns[24:47].tolist() + ['repo_name'] + ['pr_number'] + ['repo_id']
commit_data = df_pr[commit_cols].drop_duplicates()

<timed exec>:1: DtypeWarning: Columns (5,6,7,16,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
<timed exec>:2: DtypeWarning: Columns (5,6,7,16,18,19) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 1min 45s, sys: 13.6 s, total: 1min 59s
Wall time: 1min 59s


In [4]:
for col in ['created_at', 'pr_merged_at', 'pr_closed_at', 'pr_updated_at']:
    pr_data[col] = pd.to_datetime(pr_data[col])
for col in ['pr_assignees', 'pr_requested_reviewers', 'pr_requested_teams']:
    pr_data[col] = pr_data[col].apply(lambda x: literal_eval(x)).apply(lambda x: x if len(x)== 0 else [ele['id'] for ele in x])
for col in ['pr_label', 'pr_actor_id_list', 'pr_assignees_list', 'pr_requested_reviewers_list','pr_requested_teams_list', 
            'pr_actors', 'pr_commit_actors', 'all_pr_actors', 'pr_orgs', 'pr_commit_orgs']:
    pr_data[col] = pr_data[col].apply(lambda x: literal_eval(x))

pr_data['commit_actor_id_list'] = pr_data['commit_actor_id_list'].apply(lambda x: literal_eval(x) if not pd.isnull(x) else [])

In [5]:
# error correction
pr_data['pr_orgs'] = pr_data['pr_actor_id_list'].apply(lambda x: [ele.split("|")[2].strip() for ele in x if ele.split("|")[2].strip()  != 'NAN ORG'])
pr_data['pr_commit_orgs'] = pr_data['commit_actor_id_list'].apply(lambda x: [ele.split("|")[1].strip() for ele in x if ele.split("|")[1].strip() != 'NAN ORG'] if type(x) == list else [])
pr_data['all_pr_orgs'] = (pr_data['pr_orgs']+pr_data['pr_commit_orgs']).apply(lambda x: list(set([ele for ele in x if ele != 'NAN ORG'])))


In [6]:
df_pr_actor = pd.concat([pd.read_csv('data/merged_data/filtered_github_data_large/pr_actor.csv', index_col = 0),
                   pd.read_csv('data/merged_data/github_data_pre_18/pr_actor.csv', index_col = 0)])

/home/chrisliao/ipykernel_2625586/2428400343.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pr_actor = pd.concat([pd.read_csv('data/merged_data/filtered_github_data_large/pr_actor.csv', index_col = 0),


In [7]:
df_committers = pd.concat([
    commit_data[['pr_number', 'repo_name', 'repo_id', 'commit author name', 'commit author email']].drop_duplicates().rename(
        {'commit author name': 'name', 'commit author email': 'email'}, axis = 1),
    commit_data[['pr_number', 'repo_name','repo_id', 'committer name', 'commmitter email']].drop_duplicates().rename(
        {'committer name': 'name', 'commmitter email': 'email'}, axis = 1)]).dropna()
df_committers = df_committers[df_committers['name'] != "GitHub"]
df_committers = df_committers[df_committers['name'].apply(lambda x: "[bot]" not in x and '-bot' not in x)]
# known id
df_committers['actor_id'] = df_committers['email'].apply(
    lambda x: np.nan if "users.noreply.github.com" not in x or "+" not in x 
    else pd.to_numeric(x.split("+")[0].replace("\x96",""), errors = "coerce"))
df_committers['actor_login'] = df_committers['email'].apply(lambda x: x.split("@")[0].split("+")[-1] if "users.noreply.github.com" in x else np.nan)
# weird excxeption 
df_committers['actor_id'] = pd.to_numeric(df_committers['actor_id'].replace('palmtree5', 3577255).replace('user.email47079615', 47079615))
np.mean(df_committers['actor_id'].isnull())

0.9595317588296872

In [8]:
# same PR, different email, same name - use the non no-reply email
df_committers['pr_number_name'] =  df_committers['pr_number'].astype(str) + "_" + df_committers['name']
df_committers_dup = df_committers[['pr_number_name', 'actor_id']].dropna().drop_duplicates()
dict_committers_id = dict(zip(df_committers_dup['pr_number_name'], df_committers_dup['actor_id']))
df_committers['actor_id'] = df_committers.apply(lambda x: x['actor_id'] if not pd.isnull(x['actor_id']) else \
                                                dict_committers_id.get(x['pr_number_name'], np.nan), axis = 1)
df_committers.drop(['pr_number_name'], axis = 1, inplace = True)
np.mean(df_committers['actor_id'].isnull())

0.9518498282293724

In [9]:
# same repo, different email, same name
df_committers['pr_number_repo'] =  df_committers['repo_name'].astype(str) + "_" + df_committers['name']
df_committers_dup = df_committers[['pr_number_repo', 'actor_id']].dropna().drop_duplicates()
dict_committers_repo_id = dict(zip(df_committers_dup['pr_number_repo'], df_committers_dup['actor_id']))
df_committers['actor_id'] = df_committers.apply(lambda x: x['actor_id'] if not pd.isnull(x['actor_id']) else \
                                                dict_committers_repo_id.get(x['pr_number_repo'], np.nan), axis = 1)
df_committers.drop(['pr_number_repo'], axis = 1, inplace = True)
np.mean(df_committers['actor_id'].isnull())

0.910143170395012

In [10]:
# name in commit is the same as actor login
df_pr_actor['actor_login_repo_id'] = df_pr_actor['actor_login'] + "_" + df_pr_actor['repo_id'].astype(str)
df_committers['name_repo_id'] = df_committers['name'] + '_' + df_committers['repo_id'].astype(str)
dict_actors_name_repo_id = dict(zip(df_pr_actor['actor_login_repo_id'], df_pr_actor['actor_id']))
df_committers['actor_id'] = df_committers.apply(lambda x: x['actor_id'] if not pd.isnull(x['actor_id']) else \
                                                dict_actors_name_repo_id.get(x['name_repo_id'], np.nan), axis = 1)
df_committers.drop(['name_repo_id'], axis = 1, inplace = True)
np.mean(df_committers['actor_id'].isnull())

0.8101771377041967

In [11]:
df_committers['actor_id'] = pd.to_numeric(df_committers['actor_id'])

In [12]:
df_email_actor = df_committers[['email', 'actor_id']].dropna().drop_duplicates()
dict_email_actor = df_email_actor.set_index('email').to_dict()['actor_id']
df_committers['actor_id'] = df_committers.apply(lambda x: dict_email_actor.get(x['email'], np.nan) if \
                                                pd.isnull(x['actor_id']) else x['actor_id'], axis = 1)
np.mean(df_committers['actor_id'].isnull())

0.7047013898794342

In [15]:
with open('data/inputs/company_domain_match_list.yaml', 'r') as f:
    company_info = yaml.load(f, Loader=yaml.FullLoader)
df_company = pd.DataFrame(company_info)


In [16]:
df_domains = df_company[['company', 'domains']].explode('domains').drop_duplicates().groupby('domains').agg({'company':list})
df_committers_uq = df_committers[['name', 'email', 'actor_id']].drop_duplicates()
df_committers_uq['company'] = df_committers_uq['email'].apply(lambda x: [df_domains.loc[email, 'company'] for email in df_domains.index if x.endswith(email) and not x.split(email)[0][-1].isdigit() \
                                                                         and not x.split(email)[0][-1].isalpha()])
df_committers_uq['company'] = df_committers_uq['company'].apply(lambda x: "|".join(list(set([ele[0] for ele in x]))))
df_committers_uq['company'] = df_committers_uq.apply(lambda x: '' if x['email'].endswith('users.noreply.github.com') else x['company'], axis = 1)

In [17]:
# add regex stuff

In [46]:
commit_data[commit_data['commit author email'].apply(lambda x: x == 'noreply@github.com' if not pd.isnull(x) else False)]

,pr_id,commit sha,commit author name,commit author email,committer name,commmitter email,commit message,commit additions,commit deletions,commit changes total,commit files changed count,commit file changes,commit time,commit_actor_id_list,merge_day,merge_month,merge_year,closed_day,closed_month,closed_year,commit_day,commit_month,commit_year,commit parent,repo_name,pr_number,repo_id
347778,394138327,f9a91bf0d70e76e5c8cc5ded528e3ccdd8d20f38,Mehrdad,noreply@github.com,Mehrdad,noreply@github.com,Merge remote-tracking branch 'remotes/upstream...,19034.0,6267.0,25301.0,596.0,"[{'file': '.travis.yml', 'additions': 28, 'del...",2020-03-27 22:23:35,"['21118851 | NAN ORG | closed', '523212 | NAN ...",1.0,4.0,2020.0,1.0,4.0,2020.0,27.0,3.0,2020.0,NaN,ray-project/ray,7754.0,71932349
347779,394138327,5da820600ed2a9246cf4e4f91771fa357bdf58c8,Mehrdad,noreply@github.com,Mehrdad,noreply@github.com,Fix errors getting swallowed\n,2.0,2.0,4.0,1.0,"[{'file': '.travis.yml', 'additions': 2, 'dele...",2020-03-28 01:12:10,"['21118851 | NAN ORG | closed', '523212 | NAN ...",1.0,4.0,2020.0,1.0,4.0,2020.0,28.0,3.0,2020.0,NaN,ray-project/ray,7754.0,71932349
347780,394138327,2d20d87942f5d7dfe931b0e25985296ebe964b61,Mehrdad,noreply@github.com,Mehrdad,noreply@github.com,Fix find command line\n,2.0,2.0,4.0,1.0,"[{'file': 'ci/travis/test-wheels.sh', 'additio...",2020-03-28 23:28:18,"['21118851 | NAN ORG | closed', '523212 | NAN ...",1.0,4.0,2020.0,1.0,4.0,2020.0,28.0,3.0,2020.0,NaN,ray-project/ray,7754.0,71932349
347781,394138327,166ce5d12db55ccf385f5663ca5c7facdc870f5c,Mehrdad,noreply@github.com,Mehrdad,noreply@github.com,Merge remote-tracking branch 'remotes/upstream...,11449.0,8447.0,19896.0,91.0,"[{'file': 'ci/travis/test-wheels.sh', 'additio...",2020-03-29 03:44:30,"['21118851 | NAN ORG | closed', '523212 | NAN ...",1.0,4.0,2020.0,1.0,4.0,2020.0,29.0,3.0,2020.0,NaN,ray-project/ray,7754.0,71932349
347782,394138327,4c570c9838dd9b1a50d04491c26b7efc0221a18f,Mehrdad,noreply@github.com,Mehrdad,noreply@github.com,Add back accidentally removed pytest\n,2.0,2.0,4.0,1.0,"[{'file': 'ci/travis/test-wheels.sh', 'additio...",2020-03-29 03:48:07,"['21118851 | NAN ORG | closed', '523212 | NAN ...",1.0,4.0,2020.0,1.0,4.0,2020.0,29.0,3.0,2020.0,NaN,ray-project/ray,7754.0,71932349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2563122,706102325,77655363e2677312c3a0e749f401f7b928b574f5,GitHub Actions,noreply@github.com,GitHub Actions,noreply@github.com,chore: scheduled pip-compile\n,6.0,6.0,12.0,2.0,"[{'file': 'pidiff-requirements.txt', 'addition...",2021-08-08 20:30:12,['41898282 | NAN ORG | open'],NaN,NaN,NaN,NaN,NaN,NaN,8.0,8.0,2021.0,NaN,rohanpm/more-executors,282.0,124495375
803480,96851568,b5da0e5c9cc8c623fbb350dab43707d272252b4d,Mehrdad,noreply@github.com,Mehrdad,noreply@github.com,NaN,1452.0,998.0,2450.0,58.0,"[{'file': '.gitmodules', 'additions': 4, 'dele...",2020-06-20 00:29:41,['523212 | NAN ORG | closed'],NaN,NaN,NaN,2.0,6.0,2017.0,20.0,6.0,2020.0,NaN,ray-project/ray,93.0,71932349
803481,96851568,21cc593f7dfe8c6439b12c897c210040d6b9b60a,Mehrdad,noreply@github.com,Mehrdad,noreply@github.com,Merge remote-tracking branch 'upstream/master'...,369813.0,54578.0,424391.0,2864.0,"[{'file': '.bazelrc', 'additions': 102, 'delet...",2020-06-20 00:30:08,['523212 | NAN ORG | closed'],NaN,NaN,NaN,2.0,6.0,2017.0,20.0,6.0,2020.0,NaN,ray-project/ray,93.0,71932349
803482,96851568,a925a84398cde476f4ef96310696328853d20875,Mehrdad,noreply@github.com,Mehrdad,noreply@github.com,Update installation instructions for Windows\n,34.0,2.0,36.0,1.0,"[{'file': 'doc/source/installation.rst', 'addi...",2020-06-24 03:48:54,['523212 | NAN ORG | closed'],NaN,NaN,NaN,2.0,6.0,2017.0,24.0,6.0,2020.0,NaN,ray-project/ray,93.0,71932349


In [55]:
commit_data['commit author details'] = commit_data['commit author name'] + "_"+commit_data['commit author email']
author_emails_nodup = commit_data[~commit_data['commit author details'].duplicated()]
author_emails_nodup['commit_repo'] = author_emails_nodup['commit sha'] + "_" + author_emails_nodup['repo_name']
dict_author_emails = author_emails_nodup[['commit author details', 'commit_repo']].set_index('commit author details').to_dict()['commit_repo']

/home/chrisliao/ipykernel_2625586/1320579622.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  author_emails_nodup['commit_repo'] = author_emails_nodup['commit sha'] + "_" + author_emails_nodup['repo_name']


In [56]:
commit_data['committer details'] = commit_data['committer name'] + "_"+commit_data['commmitter email']
committer_emails_nodup = commit_data[~commit_data['committer details'].duplicated()]
committer_emails_nodup['commit_repo'] = committer_emails_nodup['commit sha'] + "_" + committer_emails_nodup['repo_name']
dict_committer_emails = committer_emails_nodup[['committer details', 'commit_repo']].set_index('committer details').to_dict()['commit_repo']

/home/chrisliao/ipykernel_2625586/374114196.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  committer_emails_nodup['commit_repo'] = committer_emails_nodup['commit sha'] + "_" + committer_emails_nodup['repo_name']


In [74]:
df_committers_uq['commit_repo'] = df_committers_uq.apply(
    lambda x: dict_author_emails.get(x['name']+"_"+x['email'], np.nan), axis = 1)
df_committers_uq['user_type'] = df_committers_uq['commit_repo'].apply(
    lambda x: 'author' if not pd.isnull(x) else 'committer')
df_committers_uq['commit_repo'] = df_committers_uq.apply(
    lambda x: dict_committer_emails[x['name']+"_"+x['email']] 
    if pd.isnull(x['commit_repo']) else x['commit_repo'], axis = 1)

In [95]:
username = "liaochris"
token = os.environ['token']

In [101]:
def getCommits(repo_info, sha, user_type):
    api_url = f"https://api.github.com/repos/{repo_info}/commits/{sha}"
    with requests.get(api_url, auth=(username,token)) as url:
        try:
            data = url.json()
            info = data[user_type]
            time.sleep(.75)
            if info != None:
                return [info['login'], info['id'], info['type'], info['site_admin']]
            return []
        except:
            return url['message']

In [ ]:
ncount = 1000

In [ ]:
df_committers_uq.reset_index(drop = True, inplace = True)
indices = np.array_split(df_committers_uq.index, ncount)

In [ ]:
for i in range(ncount):
    print(f"Iter {i}")
    df_committers_uq.loc[indices[i], 'committer_info'] = df_committers_uq.loc[indices[i]].apply(lambda x: getCommits(x['commit_repo'].split("_")[1], x['commit_repo'].split("_")[0],x['user_type']), axis = 1)
    df_committers_uq.to_csv('data/merged_data/committers_info.csv')